<a href="https://colab.research.google.com/github/magicWiss/ML-course/blob/Regression/Ex_02_regressione_lineare_testo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corso di Intelligenza Artificiale @ UniRoma3
## Codice Esercitazione \#2 Regressione lineare - con soluzioni

#### Parte del codice è tratto e riadattato dal seguente testo: Hands-on Machine Learning with Scikit-Learn and TensorFlow (O'Reilly book)

In [ ]:
# Controllo sulla versione di Python (≥3.5)
import sys
assert sys.version_info >= (3, 5)

In [ ]:
# Installa Scikit-Learn (≥0.20)
import sklearn
assert sklearn.__version__ >= "0.20"

Funzione di supporto per ricavare valori PIL <-> Livello di Soddisfazione, suddivisi per nazione.


In [ ]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    # seleziono solo i record con INEQUALITY = TOT
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
    # aggrego i valori 'Value' per ogni paese 'Country'
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    # rinomino il campo 2015 in GDP per capita
    gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
    # creo una tabella join in base al campo Country delle due tabelle
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                                  left_index=True, right_index=True)
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]

Impostiamo una directory di lavoro `datasets/lifesat`.

In [ ]:
import os
datapath = os.path.join("datasets", "lifesat", "")

In [ ]:
# Impostiamo i grafici in modalità inline
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
# Eseguo il download dei csv già elaborati
import urllib.request
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
os.makedirs(datapath, exist_ok=True)
for filename in ("oecd_bli_2015.csv", "gdp_per_capita.csv"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + "datasets/lifesat/" + filename
    urllib.request.urlretrieve(url, datapath + filename)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model

# Carico i DataFrame dai file csv
oecd_bli = pd.read_csv(datapath + "oecd_bli_2015.csv", thousands=',')
gdp_per_capita = pd.read_csv(datapath + "gdp_per_capita.csv",thousands=',',delimiter='\t',
                             encoding='latin1', na_values="n/a")




In [ ]:
print (oecd_bli.head())

In [ ]:
print (gdp_per_capita.head())


In [ ]:
# Ricavo i dati X,y
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

In [ ]:
# Visualizzo i dati X,y
country_stats.plot(kind='scatter', x="GDP per capita", y='Life satisfaction')
plt.show()


# *Esercizio \#1*: crea e addestra un modello lineare con funzione di costo RSS

# *Esercizio \#2*: visualizza i parametri del modello

# *Esercizio \#3*: prendi tre campioni random dal dataset e ricava la predizione in base al modello addestrato

# *Esercizio \#4*: calcola RSS MSE e RMSE valutando i tre campioni

# *Esercizio \#5*: suddividi il dataset in input in train e test con un rapporto 80/20

# *Esercizio \#6*: addestra nuovamente il modello, e ricava RSS MSE e RMSE sui test set

# *Esercizio \#7*: suddividi nuovamente il dataset ma con un rapporto 50/50. Valuta nuovamente le performance del modello e discuti eventuali differenze nei valori ottenuti.